In [ ]:
# clone codes
import os
os.chdir('/content/')

!git clone https://github.com/neuphonic/neutts-air
!apt-get install -y espeak
%pip install -r neutts-air/requirements.txt
%pip install llama-cpp-python onnxruntime
!apt-get install time ffmpeg
!/usr/bin/time -v



In [ ]:
# prepare benchmark
!git clone https://github.com/skiloop/tts-benchmarks
os.chdir('/content/tts-benchmarks/')
!DST_PATH=~/.local/bin/ sh install.sh
%pip install -r requirements.txt

In [ ]:
# test and prepare models
import os
os.chdir('/content/neutts-air/')

# Set environment variables for test text and reference files
%env TEST_TEXT=My name is Dave, and um, I'm from London
%env REF_AUDIO=./samples/dave.wav
%env REF_TEXT=./samples/dave.txt
%env REF_CODES=samples/dave.pt
%env BACKBONE=neuphonic/neutts-air-q4-gguf

# Test with different configurations
!python -m examples.onnx_example \
  --input_text "My name is Dave, and um, I'm from London" \
  --ref_codes samples/dave.pt \
  --ref_text samples/dave.txt \
  --backbone neuphonic/neutts-air-q4-gguf
!python -m examples.basic_example \
  --input_text "My name is Dave, and um, I'm from London" \
  --ref_audio ./samples/dave.wav \
  --ref_text ./samples/dave.txt \
  --backbone neuphonic/neutts-air-q4-gguf
!python -m examples.basic_example \
  --input_text "My name is Dave, and um, I'm from London" \
  --ref_audio ./samples/dave.wav \
  --ref_text ./samples/dave.txt
!python -m examples.onnx_example --input_text "My name is Dave, and um, I'm from London" --ref_audio ./samples/dave.wav --ref_text ./samples/dave.txt --ref_codes samples/dave.pt
!chartts -t "My name is Dave, and um, I'm from London" -o output.wav
# coqui tts
!tts --text "$TEST_TEXT" --out_path output.wav
!tts --test "$TEST_TEXT" --out_path output.wav --speaker_wav $REF_AUDIO --model_name tts_models/multilingual/multi-dataset

In [ ]:
# config performance test
import json
import os

# Define test parameters
TEST_TEXT = "My name is Dave, and um, I'm from London"
REF_AUDIO = "./samples/dave.wav"
REF_TEXT = "./samples/dave.txt"
REF_CODES = "samples/dave.pt"

# Build configuration using variables
config = {
  "description": "TTS性能测试",
  "iterations": 5,
  "tests": [
    {
      "name": "neutts",
      "command": f"python -m examples.basic_example --input_text \"{TEST_TEXT}\" --ref_audio {REF_AUDIO} --ref_text {REF_TEXT}"
    },
    {
      "name": "neutts-gguf",
      "command": f"python -m examples.basic_example --input_text \"{TEST_TEXT}\" --ref_audio {REF_AUDIO} --ref_text {REF_TEXT} --backbone neuphonic/neutts-air-q4-gguf"
    },
    {
      "name": "neutts-onnx-gguf",
      "command": f"python -m examples.onnx_example --input_text \"{TEST_TEXT}\" --ref_text {REF_TEXT} --ref_codes {REF_CODES} --backbone neuphonic/neutts-air-q4-gguf"
    },
    {
      "name": "neutts-onnx",
      "command": f"python -m examples.onnx_example --input_text \"{TEST_TEXT}\" --ref_text {REF_TEXT} --ref_codes {REF_CODES} --backbone neuphonic/neutts-air-q4-gguf"
    },
    {
      "name": "edge-tts",
      "command": f"edge-tts --text \"{TEST_TEXT}\" --write-media ./outputs/dave-edge-tts.wav"
    },
    {
      "name": "chartts",
      "command": f"chartts -t \"{TEST_TEXT}\" -o ./outputs/dave-chartts.wav"
    },
    {
      "name": "coqui-tts",
      "command": f"tts --text \"{TEST_TEXT}\" --out_path ./outputs/dave-coqui-tts.wav"
    }
  ]
}

with open('perftest.json', 'w') as f:
  json.dump(config, f, indent=2)


In [ ]:
# do performance test
!perftest -f perftest.json -o pefromance.json
